## I numeri di Almalibri

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from PIL import Image
from IPython.display import Image
from IPython.display import HTML, display
import matplotlib.pyplot as plt
%matplotlib inline

Il file "editori.csv" è il risultato della query che richiama l'elenco degli editori e il numero di titoli citati nelle bibliografie.

In [ ]:
df = pd.read_csv('editori.csv', 
                 names=['Editore', 'Frequenza'], header=0, skiprows=[1])

Ampiezza del catalogo dei libri adottati

In [ ]:
df.Frequenza.sum()

Il numero degli editori presenti nel catalogo

In [ ]:
df.count()

# Numero docenti e numero insegnamenti 

La query è stata lanciata nel backend. 
1. Insegnamenti singoli, ovvero contati solo una volta indipendentemente dai curricula/piani di studio in cui sono inseriti 
2. Un insegnamento può essere inserito in più corsi di laurea e in questo caso può essere contato più di una volta perché non è detto che adotti lo stesso programma nei vari corsi di laurea
3. Ci sono materie prive per definizione di programma e bibliografia. Es: TIROCINIO, TESI DI LAUREA, ecc.
4. I docenti sono stati contati per università e nome (in modo da evitare omonimie, sperando che nella stessa università non ci siano due docenti con lo stesso nome)
5. Se un docente ha più insegnamenti, come accade di sovente, è stato contato una volta sola  

ROARS riporta per il 2018, 44844 docenti tra ordinari, associati e ricercatori. A questi si devono aggiungere 31587 docenti a contratto.

In [ ]:
from IPython.display import HTML, display

data = [[2018,85387,70155,66473],
        [2019,96490,78752,76606],
        [2020,107720,81665,79954],
        ['Totali', 289597,230572,223033]
        ]
data2 = [[2018,33727,9442,43169],
        [2019,38399,12039,50438],
        [2020,39304,11091,50395],
        ['Totali',111430,32572,144002]]

display(HTML(
   '<table><tr><th>&nbsp;</th><th>Insegnamenti</th><th>Con programma</th><th>Con bibliografia</th><tr>{}</tr></table>'.format(
       '</tr><tr>'.join(
           '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in data)
       )
),
    HTML(
   '<br /><table><tr><th>&nbsp;</th><th>Docenti</th><th>Sub docenti</th><th>Totali</th><tr>{}</tr></table>'.format(
       '</tr><tr>'.join(
           '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in data2)
       )
)
       )

In [ ]:
Image(filename='almalibri-dati.jpg')

Un esempio di ricerca di adozioni per l'area 14 Scienze politiche e sociali, SSD Storia delle relazioni internazionali, per qualunque tipo di laurea. Il file csv è scaricabile direttamente da https://www.almalibri.it/ricerca/adozioni/ dopo aver effettuato la ricerca.

In [ ]:
ad = pd.read_csv('A14_SPS-06.csv', sep=";")

Eliminiamo i campi che non ci interessano.

In [ ]:
to_drop = ['laurea_tipo', 'a_a', 'laurea_classe_cod', 'curr_nome', 
           'curr_materia_anno', 'curr_materia_periodo', 'modulo_nome', 'sub_modulo_gruppo', 
           'sub_modulo_nome', 'modulo_cfu', 'modulo_periodo', 'materia_ssd_cod', 'insegnamento_prof_www',
          'traduttori', 'lingua', 'testo_uso', 'testo_freq', 'edizione', 'laurea_nome']
ad_simple = ad.drop(to_drop, axis=1)

Raggruppiamo ora le adozioni per singolo editore

In [ ]:
ad2 = ad_simple[['materia_nome','editore']].groupby(['editore'])['materia_nome'].count()\
                                .reset_index(name='numero adozioni')\
                                .sort_values('numero adozioni', ascending=False)

Disegnamo un grafico a barre con la numerosità delle adozioni; per adozioni, come è usuale, distinguiamo tutti i corsi con curricula, moduli, ecc. I dati sono ordinati per numero di adozioni e prendiamo i primi 50 editori dei 145 presenti.

In [ ]:
plt.figure(figsize=(20,10))
sns.set_theme(style="ticks", color_codes=True)
sns.barplot(x="editore", y='numero adozioni', data=ad2.head(50), color="c")
plt.xticks(rotation=90)

Associamo ora l'insegnamento all'editore e al numero di adozioni

In [ ]:
ad3 = ad[['materia_nome','editore']].groupby(['editore', 'materia_nome'])['materia_nome'].count()\
    .reset_index(name='numero adozioni').sort_values('editore')

In [ ]:
def display_table(data):
    html = "<table><tr><th>Editore</th><th>&nbsp;</th><th style=\"text-align=right\">Insegnamento</th><th>&nbsp;</th><th>Numero adozioni</th></tr>"
    for row in data:
        html += "<tr>"
        for field in row:
            html += "<td>%s<td>"%(field)
        html += "</tr>"
    html += "</table>"
    display(HTML(html))
display_table(ad3.values[:5])

Questa tabella ci mostra un'anomalia da tenere in considereazione utilizzando Almalibri. Facendo la ricerca delle adozioni per editore su almalibri.it per l'editore Alberto Longo risultano solo 15 adozioni e non 46. La ricerca sul sito infatti presenta solo un "riassunto" e non riporta le suddivisione per curricula, come invece avviene sui dati completi scaricati in formato testo.

In [ ]:
# numero di righe associate all'editore cercato
ad[ad['editore']=='Angelo Longo'].count()

## Progetto editoriale

In [ ]:
to_drop = ['a_a', 'laurea_classe_cod', 'curr_nome', 
           'curr_materia_anno', 'curr_materia_periodo', 'modulo_nome', 'sub_modulo_gruppo', 
           'sub_modulo_nome', 'modulo_cfu', 'modulo_periodo', 'materia_ssd_cod', 'insegnamento_prof_www',
          'traduttori', 'lingua', 'testo_uso', 'testo_freq', 'edizione']
ad_progetto = ad.drop(to_drop, axis=1)

In [ ]:
ad_lauree = ad.groupby(['laurea_nome','materia_nome'])

In [ ]:
ad_lauree.head

In [ ]:
ad_lauree.groups